In [1]:
%pip install -q litellm
%pip install -q matplotlib numpy pandas httpx tqdm jupyter ipywidgets aiofiles


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# %pip install uv
# %uv pip install ember-ai

In [ ]:
import os
from litellm import completion, batch_completion

from httpx import HTTPStatusError
from litellm.exceptions import NotFoundError
import pandas as pd
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from openai import OpenAI
import json
import uuid
from dotenv import load_dotenv
from litellm import turn

In [ ]:
load_dotenv()

litellm

In [11]:
range_l = 0
range_h = 1000 # 10,1000
num_samples = 2 * range_h
temp = 1
max_tokens = 2000

prompt = f"generate me a random number from [{range_l},{range_h}] inclusive. Only output a single number:"

messages=[
      {
          "role": "system",
          "content": "You are an assistant who follows the user's request exactly as they state.",
      },
      {
          "role": "user",
          "content": prompt,
      },
    ]

models = [
]

openai_models = [
    # "gpt-4.1-2025-04-14",
    # "gpt-4.1-mini-2025-04-14",
    # "gpt-4.1-nano-2025-04-14",
    # "o4-mini-2025-04-16",
    # "o3-mini-2025-01-31",
    # "o3-2025-04-16",
    # "o1-mini-2024-09-12",
    # "o1-preview-2024-09-12", # skipped
    # "gpt-4o-mini-2024-07-18",
    # "gpt-4o-2024-08-06",
    # "gpt-4-turbo-2024-04-09",
    # "gpt-4-0613",
    "gpt-3.5-turbo-0125"
    ]
anthropic_models = [
    "claude-opus-4-20250514", # skipped from overload
    "claude-sonnet-4-20250514",
    "claude-3-7-sonnet-20250219",
    "claude-3-5-sonnet-20240620", # skipped from overload
    # "claude-3-opus-20240229", # skipped from overload
    "claude-3-sonnet-20240229", # error
    "claude-3-haiku-20240307" # error
    ]
gemini_models = [
    "gemini/gemini-2.5-pro", # missing pro tier quota
    "gemini/gemini-2.5-pro-preview-06-05",
    "gemini/gemini-2.5-flash",
    "gemini/gemini-2.5-flash-preview-05-20",
    "gemini/gemini-2.5-flash-lite-preview-06-17",
    "gemini/gemini-2.0-flash"
    ]
grok_models = [
    "grok-3-latest",
    "grok-3-fast-latest",
    "grok-3-mini-beta",
    "grok-3-beta"

    ]
deepseek_models = [

  ]
other_models = [
    "llama-4"
    "qwen2.5"
    "qwen-max"
  ]

models += openai_models
models += anthropic_models
# models += gemini_models

In [12]:
# Configuration
plots_output_dir = "frequency_plots"
litellm_output_dir = "litellm"
os.makedirs(plots_output_dir, exist_ok=True)
os.makedirs(litellm_output_dir, exist_ok=True)

def parse_numbers_from_file(model, range_l, range_h):
    """Extract the actual generated numbers from the results file."""
    numbers = []
    filepath = os.path.join(litellm_output_dir, f"litellm_results_{range_l}_{range_h}_{model}.jsonl")
    
    try:
        with open(filepath, "r") as f:
            data = json.load(f)
        for entry in data:
            if isinstance(entry, list) and len(entry) >= 2:
                if isinstance(entry[1], str) and entry[1].startswith("output:"):
                    # Extract the number from "output: 42" format
                    try:
                        number_str = entry[1].replace("output: ", "").strip()
                        number = int(number_str)
                        numbers.append(number)
                    except ValueError:
                        continue  # Skip if can't parse the number
    except FileNotFoundError:
        pass
    
    return numbers

def plot_frequency(model, range_h, range_l, numbers, num_wrong):
    """Make and save a bar chart for one (model, range_h) run."""
    # full x-axis (so missing numbers appear with 0 height)
    xs = list(range(range_l, range_h + 1))
    freq_dict = pd.Series(numbers).value_counts().to_dict()
    ys = [freq_dict.get(x, 0) for x in xs]

    # choose a sensible tick step for large ranges
    width = range_h - range_l
    if   width <= 10:   step = 1
    elif width <= 100:  step = 5
    elif width <= 1000: step = 50
    else:               step = 100

    plt.figure(figsize=(8, 5))
    plt.bar(xs, ys)
    plt.xticks(xs[::step], rotation=90)        # show every *step*-th label
    plt.title(f"Number frequency\nmodel:{model}, range:[{range_l}, {range_h}], num_wrong_retry:{num_wrong}")
    plt.xlabel("Generated number")
    plt.ylabel("Frequency")
    plt.tight_layout()

    fname = f"{model.replace('/','_')}_range_{range_l}_{range_h}.png"
    plt.savefig(os.path.join(plots_output_dir, fname))
    plt.close()

def append_response_to_json(filename, *spots):
    data = []
    filepath = os.path.join(litellm_output_dir, filename)
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
    except FileNotFoundError:
        pass

    data.append(list(spots))

    with open(filepath, 'w') as file:
        json.dump(data, file, indent=2)

def get_resume_progress(results_path):
    success = 0
    num_wrong = 0
    filepath = os.path.join(litellm_output_dir, results_path)

    try:
        with open(filepath, "r") as f:
            data = json.load(f)
        for entry in data:
            if isinstance(entry, list) and len(entry) >= 2:
                if isinstance(entry[1], str) and entry[1].startswith("output:"):
                    success += 1
                elif isinstance(entry[1], str) and entry[1].startswith("wrong_output:"):
                    num_wrong += 1
    except FileNotFoundError:
        pass
    return success, num_wrong


In [14]:
for model in models:
  num_wrong = 0

  success = 0
  success, num_wrong = get_resume_progress(f"litellm_results_{range_l}_{range_h}_{model}.jsonl")
  
  pbar = tqdm(total=num_samples, initial=success, desc=f"{model}, r[{range_l},{range_h}], num_wrong={num_wrong}, num_iterations", leave=False)

  while success < num_samples and num_wrong <= 500:
    responses = batch_completion(
          model=model,
          # messages=[messages] * 100,
          messages=[messages] * (num_samples - success),
          temperature=temp,
          max_tokens=max_tokens
    )
    for response in responses:
      try:
        out_output = "output: " + str(response.choices[0].message.content)
        out_prompt_tokens = "prompt_tokens: " + str(response.usage.prompt_tokens)
        out_reasoning_tokens = "reasoning_tokens: " + str(response.usage.completion_tokens_details.reasoning_tokens) if hasattr(response.usage.completion_tokens_details, "reasoning_tokens") else "reasoning_tokens: N/A"
        out_completion_tokens = "completion_tokens: " + str(response.usage.completion_tokens)
        out_temp = "temperature: " + str(temp)
        out_messages = "messages: " + json.dumps(messages, ensure_ascii=False)

        desc = f"{model}, r[{range_l},{range_h}], {out_prompt_tokens}, {out_completion_tokens}, {out_reasoning_tokens}, num_wrong={num_wrong}"
        pbar.set_description(desc)
        
        append_response_to_json(f"litellm_{range_l}_{range_h}_{model}.jsonl", model, out_prompt_tokens, out_reasoning_tokens, out_completion_tokens, out_temp, out_messages)
        number = int(response.choices[0].message.content)

        append_response_to_json(f"litellm_results_{range_l}_{range_h}_{model}.jsonl", model, out_output)

        success += 1
        pbar.update(1)
      except ValueError:
        num_wrong += 1
        out_wrong = "wrong_output: " + str(response.choices[0].message.content)
        append_response_to_json(f"litellm_results_{range_l}_{range_h}_{model}.jsonl", model, out_wrong)
        pbar.set_description(f"{model}, r[{range_l},{range_h}], num_wrong={num_wrong}")
  # pbar.close()
  numbers = parse_numbers_from_file(model, range_l, range_h)
  plot_frequency(model, range_h, range_l, numbers, num_wrong) 

gpt-3.5-turbo-0125, r[0,1000], num_wrong=510, num_iterations:  99%|#########9| 1986/2000 [00:00<?, ?it/s]

claude-opus-4-20250514, r[0,1000], num_wrong=0, num_iterations: 100%|##########| 2000/2000 [00:00<?, ?it/s]

claude-sonnet-4-20250514, r[0,1000], num_wrong=0, num_iterations: 100%|##########| 2000/2000 [00:00<?, ?it/s]

claude-3-7-sonnet-20250219, r[0,1000], num_wrong=0, num_iterations: 100%|##########| 2000/2000 [00:00<?, ?it/s…

claude-3-5-sonnet-20240620, r[0,1000], num_wrong=8, num_iterations: 100%|##########| 2000/2000 [00:00<?, ?it/s…

claude-3-sonnet-20240229, r[0,1000], num_wrong=0, num_iterations: 100%|##########| 2000/2000 [00:00<?, ?it/s]

claude-3-haiku-20240307, r[0,1000], num_wrong=0, num_iterations:  45%|####5     | 900/2000 [00:00<?, ?it/s]

foundry 

In [ ]:
# Configuration
range_l = 0
range_h = 1000 # 1000
num_samples = 2 * range_h
temp = 1
max_tokens = 2000

models = [
    # "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
    "meta-llama/Llama-4-Scout-17B-16E-Instruct",
    # "meta-llama/Meta-Llama-3.1-8B-Instruct",
    # "Qwen/Qwen2.5-72B-Instruct"
]

prompt = f"generate me a random number from [{range_l},{range_h}] inclusive. Only output a single number:"

messages=[
      {
          "role": "system",
          "content": "You are an assistant who follows the user's request exactly as they state.",
      },
      {
          "role": "user",
          "content": prompt,
      },
    ]

In [ ]:
# Directory setup
import time


plots_output_dir = "frequency_plots"
foundry_output_dir = "foundry"
os.makedirs(plots_output_dir, exist_ok=True)
os.makedirs(foundry_output_dir, exist_ok=True)

client = OpenAI(
    base_url="https://avior.mlfoundry.com/v1",
    api_key=os.getenv("FOUNDRY_API_KEY"),
)

# Directory setup
import time
import os
import json
import uuid
from openai import OpenAI
import pandas as pd
import matplotlib.pyplot as plt

plots_output_dir = "frequency_plots"
foundry_output_dir = "foundry"
os.makedirs(plots_output_dir, exist_ok=True)
os.makedirs(foundry_output_dir, exist_ok=True)

client = OpenAI(
    base_url="https://avior.mlfoundry.com/v1",
    api_key=os.getenv("FOUNDRY_API_KEY"),
)

def plot_frequency(model, range_h, range_l, numbers, num_wrong):
    """Make and save a bar chart for one (model, range_h) run."""
    # full x-axis (so missing numbers appear with 0 height)
    xs = list(range(range_l, range_h + 1))
    freq_dict = pd.Series(numbers).value_counts().to_dict()
    ys = [freq_dict.get(x, 0) for x in xs]

    # choose a sensible tick step for large ranges
    width = range_h - range_l
    if   width <= 10:   step = 1
    elif width <= 100:  step = 5
    elif width <= 1000: step = 50
    else:               step = 100

    plt.figure(figsize=(8, 5))
    plt.bar(xs, ys)
    plt.xticks(xs[::step], rotation=90)        # show every *step*-th label
    plt.title(f"Number frequency\nmodel:{model}, range:[{range_l}, {range_h}], num_wrong_retry:{num_wrong}")
    plt.xlabel("Generated number")
    plt.ylabel("Frequency")
    plt.tight_layout()

    fname = f"{model.replace('/','_')}_range_{range_l}_{range_h}.png"
    plt.savefig(os.path.join(plots_output_dir, fname))
    plt.close()

def create_batch_file(model, num_samples, range_l, range_h, temp, max_tokens):
    """Create a batch file for the given model and parameters."""
    prompt = f"generate me a random number from [{range_l},{range_h}] inclusive. Only output a single number:"
    batch_filename = f"foundrybatch_{range_l}_{range_h}_{model.replace('/', '_')}.jsonl"
    batch_filepath = os.path.join(foundry_output_dir, batch_filename)
    
    with open(batch_filepath, "w") as f:
        for i in range(num_samples):
            batch_request = {
                "custom_id": f"{model}-{uuid.uuid4()}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": model,
                    "messages": messages,
                    "max_tokens": max_tokens,
                    "min_tokens": 0,
                    "temperature": temp,
                },
            }
            json.dump(batch_request, f)
            f.write("\n")

    return batch_filepath

def submit_batch_job(batch_filepath):
    """Submit a batch job and return the job ID."""
    batch_file = client.files.create(
        file=open(batch_filepath, "rb"),
        purpose="batch",
    )
    
    print(f"New batch file created: {batch_file}")
    
    batch_job = client.batches.create(
        input_file_id=batch_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
    )
    
    print(f"New batch job created: {batch_job}")
    return batch_job.id

def check_batch_status(batch_job_id):
    """Check the status of a batch job."""
    batch_status = client.batches.retrieve(batch_job_id)
    return batch_status.status, batch_status

def download_batch_results(batch_job_id, model):
    """Download and save batch results."""
    completed_batch = client.batches.retrieve(batch_job_id)
    
    if not completed_batch.output_file_id:
        print(f"No output file for batch {batch_job_id}")
        return None
    
    output_file = client.files.content(completed_batch.output_file_id)
    results_filename = f"foundrybatch_results_{range_l}_{range_h}_{model.replace('/', '_')}.jsonl"
    results_filepath = os.path.join(foundry_output_dir, results_filename)
    
    with open(results_filepath, "w") as f:
        f.write(output_file.text)
    
    print(f"Results saved to {results_filepath}")
    return results_filepath

def parse_batch_results(results_filepath):
    """Parse batch results and extract numbers."""
    numbers = []
    num_wrong = 0
    
    try:
        with open(results_filepath, "r") as f:
            for line in f:
                if line.strip():
                    result = json.loads(line)
                    try:
                        content = result['response']['choices'][0]['message']['content']
                        number = int(content.strip())
                        numbers.append(number)
                    except (ValueError, KeyError):
                        num_wrong += 1
                        print(f"Could not parse: {content if 'content' in locals() else 'unknown'}")
    except FileNotFoundError:
        print(f"Results file not found: {results_filepath}")
    
    return numbers, num_wrong

In [52]:
# Phase 1: Submit all batch jobs
print("=== Phase 1: Submitting all batch jobs ===")
batch_jobs = {}  # {model: batch_job_id}

for model in models:
    print(f"\n--- Submitting batch for model: {model} ---")
    
    # Create batch file
    batch_filepath = create_batch_file(model, num_samples, range_l, range_h, temp, max_tokens)
    
    # Submit batch job
    batch_job_id = submit_batch_job(batch_filepath)
    batch_jobs[model] = batch_job_id
    
    print(f"Submitted batch job {batch_job_id} for model {model}")

print(f"\n=== All {len(batch_jobs)} batch jobs submitted ===")

=== Phase 1: Submitting all batch jobs ===

--- Submitting batch for model: meta-llama/Llama-4-Scout-17B-16E-Instruct ---
New batch file created: FileObject(id='56039d39-20b8-44a8-8359-5c6dc5cf067d', bytes=9018, created_at='2025-07-19T19:14:19.653850+00:00', filename='1foundrybatch_0_1000_meta-llama_Llama-4-Scout-17B-16E-Instruct.jsonl', object=None, purpose=None, status=None, expires_at=None, status_details=None)
New batch job created: Batch(id='056b4b0a-7fd7-48fa-b5ad-286b22657f6d', completion_window='24h', created_at=1752952460, endpoint='/v1/chat/completions', input_file_id='56039d39-20b8-44a8-8359-5c6dc5cf067d', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1753038860, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
Submitted batch job 056b4b0a-7fd7-48fa-b5ad-286b2

In [53]:
# Phase 2: Check status and download available results
print("\n=== Phase 2: Checking status and downloading available results ===")
completed_models = []
failed_models = []
pending_models = []

for model, batch_job_id in batch_jobs.items():
    status, batch_info = check_batch_status(batch_job_id)
    print(f"  {model}: {status}")
    
    if status == 'completed':
        print(f"\n--- Processing completed model: {model} ---")
        
        # Download results
        results_filepath = download_batch_results(batch_job_id, model)
        
        if results_filepath:
            # Parse results
            numbers, num_wrong = parse_batch_results(results_filepath)
            
            print(f"Completed: {len(numbers)} successful, {num_wrong} failed")
            
            # Generate plot
            plot_frequency(model, range_h, range_l, numbers, num_wrong)
            completed_models.append(model)
        else:
            print(f"Failed to download results for model {model}")
            failed_models.append(model)
            
    elif status == 'failed':
        print(f"Batch failed for model {model}: {batch_info.errors}")
        failed_models.append(model)
    else:
        pending_models.append(model)

print(f"\n=== Batch job status summary ===")
print(f"Completed: {len(completed_models)} models")
print(f"Failed: {len(failed_models)} models") 
print(f"Still pending: {len(pending_models)} models")

if completed_models:
    print(f"Successfully completed: {completed_models}")
if failed_models:
    print(f"Failed models: {failed_models}")
if pending_models:
    print(f"Pending models: {pending_models}")
    print("Re-run this script later to check for completed jobs.")


=== Phase 2: Checking status and downloading available results ===
  meta-llama/Llama-4-Scout-17B-16E-Instruct: completed

--- Processing completed model: meta-llama/Llama-4-Scout-17B-16E-Instruct ---
Results saved to foundry/1foundrybatch_results_0_1000_meta-llama_Llama-4-Scout-17B-16E-Instruct.jsonl
Completed: 18 successful, 0 failed

=== Batch job status summary ===
Completed: 1 models
Failed: 0 models
Still pending: 0 models
Successfully completed: ['meta-llama/Llama-4-Scout-17B-16E-Instruct']


In [ ]:

# test_path = [
#     ("meta-llama/Meta-Llama-3.1-8B-Instruct", "foundry/foundrybatch_results_0_1000_meta-llama_Meta-Llama-3.1-8B-Instruct.jsonl"),
# ]
# for model, results_filepath in test_path:
#     numbers, num_wrong = parse_batch_results(results_filepath)
#     print(f"Completed: {len(numbers)} successful, {num_wrong} failed")
#     # Generate plot
#     plot_frequency(model, range_h, range_l, numbers, num_wrong)

Could not parse: 853.
Could not parse: 854.
Could not parse: 948.
Could not parse: 467.
Could not parse: 846.
Could not parse: 854.
Could not parse: 913.
Could not parse: 418.
Could not parse: 657.
Could not parse: 871.
Could not parse: 876.
Could not parse: 817.
Completed: 2000 successful, 12 failed


consolidate all files (autogenerated)

In [58]:
import os
import json
import glob
import re
from typing import List, Dict, Any, Tuple, Optional

def extract_number_from_output(output_str: str) -> Optional[int]:
    """Extract number from output string, handling both 'output: X' and 'wrong_output: X' formats."""
    if output_str.startswith("output: "):
        try:
            return int(output_str.replace("output: ", "").strip())
        except ValueError:
            return None
    elif output_str.startswith("wrong_output: "):
        # Try to extract number from wrong output (might be float or text)
        wrong_content = output_str.replace("wrong_output: ", "").strip()
        try:
            # Try to convert to float first, then to int
            return int(float(wrong_content))
        except ValueError:
            # If it's not a number, skip it
            return None
    return None

def extract_model_name_from_custom_id(custom_id: str) -> str:
    """Extract model name from custom_id, handling various formats."""
    # Split by dashes
    parts = custom_id.split('-')
    
    # UUID is typically the last 5 parts (8-4-4-4-12 characters)
    # Look for UUID pattern at the end
    if len(parts) >= 5:
        # Check if last 5 parts look like UUID
        uuid_parts = parts[-5:]
        
        if (len(uuid_parts[0]) == 8 and len(uuid_parts[1]) == 4 and 
            len(uuid_parts[2]) == 4 and len(uuid_parts[3]) == 4 and 
            len(uuid_parts[4]) == 12):
            # Remove UUID parts
            model_parts = parts[:-5]
        else:
            # Fallback: assume last part is UUID
            model_parts = parts[:-1]
    else:
        # Not enough parts for UUID, use all
        model_parts = parts
    
    # Reconstruct model name
    model_name = '-'.join(model_parts)
    
    # Handle specific model name formats
    if model_name.startswith('meta-llama-'):
        # Convert meta-llama-Model to meta-llama/Model
        model_name = model_name.replace('meta-llama-', 'meta-llama/', 1)
    elif model_name.startswith('meta') and 'llama' in model_name:
        # Handle other meta-llama variations
        parts = model_name.split('-')
        if len(parts) >= 2 and parts[0] == 'meta' and parts[1] == 'llama':
            model_name = 'meta-llama/' + '-'.join(parts[2:])
    
    return model_name if model_name else "unknown"

def process_litellm_file(filepath: str) -> List[Tuple[int, str]]:
    """Process a LiteLLM results file and extract (number, model) pairs."""
    results = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        for item in data:
            if len(item) >= 2:
                model_name = item[0]
                output_str = item[1]
                number = extract_number_from_output(output_str)
                if number is not None:
                    results.append((number, model_name))
                else:
                    print(f"Skipping invalid output in {filepath}: {output_str}")
    
    except Exception as e:
        print(f"Error processing LiteLLM file {filepath}: {e}")
    
    return results

def process_foundry_results_file(filepath: str) -> List[Tuple[int, str]]:
    """Process a Foundry batch results file (JSONL format) and extract (number, model) pairs."""
    results = []
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if line:
                    try:
                        result = json.loads(line)
                        # Extract model from the result
                        if 'response' in result and 'choices' in result['response']:
                            content = result['response']['choices'][0]['message']['content']
                            # Extract model name from custom_id
                            model_name = "unknown"
                            if 'custom_id' in result:
                                model_name = extract_model_name_from_custom_id(result['custom_id'])
                            
                            try:
                                number = int(content.strip())
                                results.append((number, model_name))
                            except ValueError:
                                print(f"Could not parse number from foundry result in {filepath}, line {line_num}: {content}")
                    
                    except json.JSONDecodeError as e:
                        print(f"Error parsing line {line_num} in {filepath}: {e}")
    
    except Exception as e:
        print(f"Error processing Foundry results file {filepath}: {e}")
    
    return results

def find_matching_files(base_pattern: str, folders: List[str]) -> List[str]:
    """Find files matching the pattern in the specified folders."""
    found_files = []
    for folder in folders:
        if os.path.exists(folder):
            # Use more specific patterns to avoid cross-contamination
            if base_pattern == "0_10":
                # Match files that end with _0_10_ followed by model name
                pattern = os.path.join(folder, f"*_0_10_*")
            elif base_pattern == "0_1000":
                # Match files that end with _0_1000_ followed by model name
                pattern = os.path.join(folder, f"*_0_1000_*")
            else:
                # Fallback to original pattern
                pattern = os.path.join(folder, f"*{base_pattern}*")
            
            files = glob.glob(pattern)
            
            # Additional filtering to ensure exact match
            filtered_files = []
            for file in files:
                filename = os.path.basename(file)
                if base_pattern == "0_10":
                    # Must contain _0_10_ and NOT _0_1000_
                    if "_0_10_" in filename and "_0_1000_" not in filename:
                        filtered_files.append(file)
                elif base_pattern == "0_1000":
                    # Must contain _0_1000_
                    if "_0_1000_" in filename:
                        filtered_files.append(file)
                else:
                    filtered_files.append(file)
            
            found_files.extend(filtered_files)
        else:
            print(f"Warning: Folder {folder} does not exist")
    return found_files

def consolidate_files(range_pattern: str, output_filename: str):
    """Consolidate LiteLLM and Foundry files for a specific range pattern."""
    print(f"\nProcessing files for pattern: {range_pattern}")
    
    # Find LiteLLM files with exact pattern matching
    litellm_pattern = f"litellm_results_{range_pattern}"
    litellm_files = find_matching_files(range_pattern, ["litellm", "."])
    # Filter to only litellm files
    litellm_files = [f for f in litellm_files if "litellm_results_" in os.path.basename(f)]
    print(f"Found LiteLLM files: {litellm_files}")
    
    # Find Foundry results files with exact pattern matching
    foundry_pattern = f"foundrybatch_results_{range_pattern}"
    foundry_files = find_matching_files(range_pattern, ["foundry", "."])
    # Filter to only foundry results files
    foundry_files = [f for f in foundry_files if "foundrybatch_results_" in os.path.basename(f)]
    print(f"Found Foundry files: {foundry_files}")
    
    all_results = []
    
    # Process LiteLLM files
    for filepath in litellm_files:
        print(f"Processing LiteLLM file: {filepath}")
        results = process_litellm_file(filepath)
        all_results.extend(results)
        print(f"  Extracted {len(results)} valid results")
    
    # Process Foundry files
    for filepath in foundry_files:
        print(f"Processing Foundry file: {filepath}")
        results = process_foundry_results_file(filepath)
        all_results.extend(results)
        print(f"  Extracted {len(results)} valid results")
    
    # Write consolidated results to JSONL
    if all_results:
        os.makedirs(os.path.dirname(output_filename) if os.path.dirname(output_filename) else '.', exist_ok=True)
        with open(output_filename, 'w', encoding='utf-8') as f:
            for number, model in all_results:
                json.dump([number, model], f)
                f.write('\n')
        
        print(f"Consolidated {len(all_results)} results into {output_filename}")
        
        # Show some statistics
        unique_models = set(model for _, model in all_results)
        print(f"  Total responses: {len(all_results)}")
        print(f"  Unique models: {len(unique_models)}")
        print(f"  Models found: {sorted(unique_models)}")
        
    else:
        print(f"No results found for pattern {range_pattern}")
        
    return all_results

# Run the consolidation
print("JSONL File Consolidation Script")
print("=" * 40)

# Define the patterns to consolidate
patterns_to_consolidate = [
    ("0_1000", "model_responses_0_1000.jsonl"),
    ("0_10", "model_responses_0_10.jsonl")
]

for range_pattern, output_file in patterns_to_consolidate:
    consolidate_files(range_pattern, output_file)

print("\nConsolidation complete!")

# Optional: Display the first few lines of each output file to verify
for _, output_file in patterns_to_consolidate:
    if os.path.exists(output_file):
        print(f"\nFirst 5 lines of {output_file}:")
        with open(output_file, 'r') as f:
            for i, line in enumerate(f):
                if i >= 5:
                    break
                print(f"  {line.strip()}")
    else:
        print(f"\nFile {output_file} was not created (no matching input files found)")

JSONL File Consolidation Script

Processing files for pattern: 0_1000
Found LiteLLM files: ['litellm/litellm_results_0_1000_gpt-4.1-mini-2025-04-14.jsonl', 'litellm/litellm_results_0_1000_claude-3-5-sonnet-20240620.jsonl', 'litellm/litellm_results_0_1000_gpt-3.5-turbo-0125.jsonl', 'litellm/litellm_results_0_1000_gpt-4.1-2025-04-14.jsonl', 'litellm/litellm_results_0_1000_claude-opus-4-20250514.jsonl', 'litellm/litellm_results_0_1000_claude-sonnet-4-20250514.jsonl', 'litellm/litellm_results_0_1000_claude-3-sonnet-20240229.jsonl', 'litellm/litellm_results_0_1000_gpt-4o-2024-08-06.jsonl', 'litellm/litellm_results_0_1000_o3-mini-2025-01-31.jsonl', 'litellm/litellm_results_0_1000_claude-3-haiku-20240307.jsonl', 'litellm/litellm_results_0_1000_gpt-4-0613.jsonl', 'litellm/litellm_results_0_1000_gpt-4-turbo-2024-04-09.jsonl', 'litellm/litellm_results_0_1000_gpt-4o-mini-2024-07-18.jsonl', 'litellm/litellm_results_0_1000_o1-preview-2024-09-12.jsonl', 'litellm/litellm_results_0_1000_claude-3-opus-